In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [12]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [17]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

19054

In [18]:
total = 0
for obj in qs:
    total += obj.close_price

In [19]:
total / qs.count()

Decimal('301.3180998950351632203211924')

In [20]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 3, 'avg_price': Decimal('578.0588562763268745')}, {'company': 2, 'avg_price': Decimal('172.1261418694690265')}, {'company': 7, 'avg_price': Decimal('420.8373171088251218')}, {'company': 1, 'avg_price': Decimal('229.1055368381618382')}, {'company': 8, 'avg_price': Decimal('140.8298014596793491')}]>

In [24]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [25]:
rolling_qs

[<StockQuote: StockQuote object (6466357)>,
 <StockQuote: StockQuote object (6466358)>,
 <StockQuote: StockQuote object (6466359)>,
 <StockQuote: StockQuote object (6466360)>,
 <StockQuote: StockQuote object (6466361)>]

In [29]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 6466357 226.4512 226.4512
2 6466358 226.3638 226.4075
3 6466359 225.9787 226.2645666666666666666666667
4 6466360 225.9750 226.192175
5 6466361 225.9900 226.15174


In [30]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [39]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [40]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

6466361 225.9900 226.1517400000000000 2024-11-13 18:40:00+00:00
6466360 225.9750 226.2457400000000000 2024-11-13 18:35:00+00:00
6466359 225.9787 226.3627400000000000 2024-11-13 18:30:00+00:00
6466358 226.3638 226.4130000000000000 2024-11-13 18:25:00+00:00
6466357 226.4512 226.4692400000000000 2024-11-13 18:20:00+00:00


In [38]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(6466361,
 Decimal('225.9900'),
 Decimal('226.1517400000000000'),
 Decimal('225.9005650000000000'))